In [1]:
import logging
import os
import dspy
import gqr
import pandas as pd

from pathlib import Path
from time import perf_counter
from datetime import datetime
from tqdm import tqdm

from gqr.core.evaluator import Evaluator, evaluate, evaluate_by_dataset
from util import Classify, SafePredict, build_examples, metric, score_program

logging.getLogger("dspy.optimizers.bootstrap_fewshot").setLevel(logging.WARNING)
logging.getLogger("dspy.evaluate.evaluate").setLevel(logging.INFO)

/storage/mirrored/sleher_richard/Desktop/gqr-llm-benchmark/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

## Models
Run DSPy + BootstrapFewShot baselines for the selected models only.


In [3]:
models_to_test = [
    "mistral:7b",
    "granite3.3:2b",
    "phi4:14b",
    "qwen3:14b",
]

In [4]:
# Load training dataset for model development
train_data, eval_data = gqr.load_train_dataset()

## Run BootstrapFewShot 
Optimize with BootstrapFewShot, then evaluate on ID/OOD splits.


In [5]:
result_columns = ["model", "avg_latency", "id_acc", "ood_acc", "gqr_score", "dataset_acc"]

results = []
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
results_dir = Path("../results")
results_dir.mkdir(exist_ok=True)
output_file = results_dir / f"fewshot_results_{timestamp}.csv"

trainset = build_examples(train_data)[:300]

for model_name in models_to_test:
    print()
    print("=" * 60)
    print(f"DSPy + BootstrapFewShot: {model_name}")
    print("=" * 60)
    try:
        lm = dspy.LM(f"ollama_chat/{model_name}", 
                    api_base="http://localhost:11434", 
                    cache=False, 
                    reasoning_effort='low' if model_name.startswith('gpt-oss') else False,
                    
                )
        dspy.configure(lm=lm)

        student = SafePredict(Classify)
        optimizer = dspy.BootstrapFewShotWithRandomSearch(
            max_labeled_demos=6,
            num_candidate_programs=4,
            metric=metric,
        )
        optimized_program = optimizer.compile(student, trainset=trainset)
        saves_dir = Path("../saves")
        saves_dir.mkdir(exist_ok=True)
        program_path = saves_dir / f"fewshot_{model_name.replace(':', '_')}.json"
        optimized_program.save(program_path)
        print(f"Saved program to: {program_path}")

        prompt_latencies = []
        current_pbar = None

        def timed_score(text: str) -> int:
            start = perf_counter()
            prediction = score_program(text, program=optimized_program)
            prompt_latencies.append(perf_counter() - start)
            if current_pbar is not None:
                current_pbar.update(1)
            return prediction

        id_test_data = gqr.load_id_test_dataset()
        ood_test_data = gqr.load_ood_test_dataset()
        total_size = len(id_test_data) + len(ood_test_data)

        with tqdm(total=total_size, desc=f"{model_name}") as pbar:
            current_pbar = pbar
            id_test_data["predictions"] = [timed_score(doc) for doc in id_test_data["text"].values]
            ood_test_data["predictions"] = [timed_score(doc) for doc in ood_test_data["text"].values]

        id_scores = evaluate(
            predictions=id_test_data["predictions"],
            ground_truth=id_test_data["label"],
        )
        id_acc = id_scores["accuracy"]

        ood_overall_scores = Evaluator.evaluate(
            predicted_labels=ood_test_data["predictions"],
            true_labels=ood_test_data["label"],
        )

        ood_scores_df = evaluate_by_dataset(
            ood_test_data, pred_col="predictions", true_col="label", dataset_col="dataset"
        )
        if ood_scores_df.empty:
            ood_acc = ood_overall_scores["accuracy"]
            dataset_acc = {}
        else:
            ood_acc = ood_scores_df["accuracy"].mean()
            dataset_acc = dict(zip(ood_scores_df["dataset"], ood_scores_df["accuracy"]))

        gqr_score = 2 * (id_acc * ood_acc) / (id_acc + ood_acc) if (id_acc + ood_acc) > 0 else 0.0
        avg_latency = sum(prompt_latencies) / len(prompt_latencies) if prompt_latencies else None
        latency_display = f"{avg_latency:.3f}s" if avg_latency is not None else "n/a"

        result = {
            "model": model_name,
            "avg_latency": avg_latency,
            "id_acc": id_acc,
            "ood_acc": ood_acc,
            "gqr_score": gqr_score,
            "dataset_acc": str(dataset_acc),
        }
        results.append(result)

        print()
        print(
            f"ID: {id_acc:.4f} | OOD: {ood_acc:.4f} | GQR: {gqr_score:.4f} | Latency: {latency_display}"
        )
        print(f"Per-dataset: {dataset_acc}")
        pd.DataFrame(results, columns=result_columns).to_csv(output_file, index=False)
        print(f"Saved to: {output_file}")
    except Exception as e:
        print(f"Failed: {e}")
        results.append({
            "model": model_name,
            "avg_latency": None,
            "id_acc": None,
            "ood_acc": None,
            "gqr_score": None,
            "dataset_acc": None,
        })
        pd.DataFrame(results, columns=result_columns).to_csv(output_file, index=False)

print()
print("=" * 60)
print("BootstrapFewShot runs complete!")
print("=" * 60)


DSPy + BootstrapFewShot: mistral:7b
Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 4 candidate sets.
Average Metric: 259.00 / 300 (86.3%): 100%|██████████| 300/300 [01:00<00:00,  4.94it/s]

2025/12/30 11:26:48 INFO dspy.evaluate.evaluate: Average Metric: 259 / 300 (86.3%)



New best score: 86.33 for seed -3
Scores so far: [86.33]
Best score so far: 86.33
Average Metric: 283.00 / 300 (94.3%): 100%|██████████| 300/300 [01:24<00:00,  3.56it/s]

2025/12/30 11:28:12 INFO dspy.evaluate.evaluate: Average Metric: 283 / 300 (94.3%)



New best score: 94.33 for seed -2
Scores so far: [86.33, 94.33]
Best score so far: 94.33


...

Bootstrapped 2 full traces after 2 examples for up to 1 rounds, amounting to 2 attempts.
Average Metric: 268.00 / 300 (89.3%): 100%|██████████| 300/300 [00:58<00:00,  5.15it/s]

2025/12/30 20:57:46 INFO dspy.evaluate.evaluate: Average Metric: 268 / 300 (89.3%)



Scores so far: [94.67, 95.0, 95.33, 96.67, 95.67, 96.67, 89.33]
Best score so far: 96.67
7 candidate programs found.
Saved program to: ../saves/fewshot_qwen3_14b.json


qwen3:14b: 100%|██████████| 22457/22457 [2:25:38<00:00,  2.57it/s]  


ID: 0.9637 | OOD: 0.9198 | GQR: 0.9412 | Latency: 0.389s
Per-dataset: {'jigsaw': 0.9281269446172993, 'olid': 0.8104651162790698, 'hate_xplain': 0.9435551811288964, 'hate_speech_slovak ': 0.9238790406673618, 'dkhate': 0.9240121580547113, 'web_questions': 0.9084645669291339, 'ml_questions': 1.0}
Saved to: ../results/fewshot_results_20251230_112547.csv

BootstrapFewShot runs complete!
